# 사전 학습된 ConvNet을 이용한 전이 학습
# tensorflowr -> pytorch


이 튜토리얼에서는 사전 훈련된 네트워크에서 전이 학습을 사용하여 고양이와 개의 이미지를 분류하는 방법을 배우게 됩니다.

사전 훈련된 모델은 이전에 대규모 데이터셋에서 훈련된 저장된 네트워크로, 일반적으로 대규모 이미지 분류 작업에서 훈련된 것입니다. 사전 훈련된 모델을 그대로 사용하거나 전이 학습을 사용하여 이 모델을 주어진 작업으로 사용자 정의하세요.

이미지 분류를 위한 전이 학습을 직관적인 시각에서 바라보면 모델이 충분히 크고 일반적인 데이터 집합에서 훈련된다면, 이 모델은 사실상 시각 세계의 일반적인 모델로서 기능할 것이라는 점입니다. 그런 다음 대규모 데이터셋에서 대규모 모델을 교육하여 처음부터 시작할 필요 없이 이러한 학습된 특징 맵을 활용할 수 있습니다.

이번 notebook에서는 사전 훈련된 모델을 사용자 정의하는 두 가지 방법을 시도 해보겠습니다.:

1. 특성 추출: 새 샘플에서 의미 있는 특성을 추출하기 위해 이전 네트워크에서 학습한 표현을 사용합니다. 사전 훈련된 모델 위에 처음부터 훈련할 새 분류자를 추가하기만 하면 이전에 데이터세트로 학습한 특성 맵의 용도를 재사용할 수 있습니다.

전체 모델을 재훈련시킬 필요는 없습니다. 기본 컨볼루션 네트워크에는 그림 분류에 일반적으로 유용한 기능이 이미 포함되어 있습니다. 그러나 사전 훈련된 모델의 최종 분류 부분은 기존의 분류 작업에 따라 다르며 이후에 모델이 훈련된 클래스 집합에 따라 다릅니다.

1. 미세 조정: 고정된 기본 모델의 일부 최상위 층을 고정 해제하고 새로 추가 된 분류기 층과 기본 모델의 마지막 층을 함께 훈련시킵니다. 이를 통해 기본 모델에서 고차원 특징 표현을 "미세 조정"하여 특정 작업에 보다 관련성이 있도록 할 수 있습니다.

일반적인 기계 학습 일련의 과정을 진행합니다.

1. 데이터 검사 및 이해
2. 입력 파이프 라인 빌드(이 경우 Keras ImageDataGenerator를 사용)
3. 모델 작성
    - 사전 훈련된 기본 모델(또한 사전 훈련된 가중치)에 적재
    - 분류 레이어를 맨 위에 쌓기
4. 모델 훈련
5. 모델 평가


In [1]:
import os
import zipfile
import urllib.request
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
import torchvision
from torchvision import transforms, datasets, models

## 데이터 전처리

### 데이터 다운로드

이 튜토리얼에서는 수천 개의 고양이와 개의 이미지가 포함된 데이터세트를 사용합니다. 이미지가 포함된 zip 파일을 다운로드하여 추출은 다음 `tf.keras.utils.image_dataset_from_directory` 유틸리티를 사용하여 훈련 및 검증을 위한 `tf.data.Dataset`를 생성합니다. 이 [튜토리얼](https://www.tensorflow.org/tutorials/load_data/images)에서 이미지 로드에 대해 자세히 알아볼 수 있습니다.

In [2]:
# 1. 데이터셋 다운로드 및 압축 해제 (이미 다운로드된 경우 건너뜁니다)
url = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
zip_filename = 'cats_and_dogs_filtered.zip'
if not os.path.exists('cats_and_dogs_filtered'):
    print("Downloading dataset...")
    urllib.request.urlretrieve(url, zip_filename)
    with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
        zip_ref.extractall('.')
    print("Dataset downloaded and extracted.")

Dataset downloaded and extracted.


In [3]:
# 2. 경로 설정
BASE_DIR = os.path.join(os.getcwd(), 'cats_and_dogs_filtered')
train_dir = os.path.join(BASE_DIR, 'train')
validation_dir = os.path.join(BASE_DIR, 'validation')

In [4]:
# 3. 하이퍼파라미터 설정
BATCH_SIZE = 32
IMG_SIZE = (160, 160)  # (height, width)
base_learning_rate = 0.0001
initial_epochs = 10
fine_tune_epochs = 10

In [5]:
# 4. 데이터 전처리 및 데이터 증강
# TF 코드에서는 RandomFlip, RandomRotation을 별도 레이어로 사용합니다.
# 여기서는 transforms를 이용하여 이미지 증강 및 전처리 수행합니다.
# MobileNetV2의 TF preprocess_input는 [0,255] -> [-1,1] 변환을 수행하므로,
# ToTensor() 후 Normalize(mean=0.5, std=0.5)로 동일 효과를 줍니다.

train_transforms = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.RandomHorizontalFlip(),
    # tf.keras.layers.RandomRotation(0.2)는 약 ±72도 회전 효과를 줍니다.
    transforms.RandomRotation(72),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

val_transforms = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])